In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [11]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

In [12]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
train.shape

(891, 12)

In [14]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [15]:
train = train.drop(labels=["PassengerId","Ticket","Cabin","Name"],axis=1)
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [17]:
train = train.dropna()

In [19]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [20]:
train_dummy = pd.get_dummies(train[["Sex","Embarked"]])

In [21]:
train_dummy

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,1,0,0,1
1,1,0,1,0,0
2,1,0,0,0,1
3,1,0,0,0,1
4,0,1,0,0,1
...,...,...,...,...,...
885,1,0,0,1,0
886,0,1,0,0,1
887,1,0,0,0,1
889,0,1,1,0,0


In [22]:
train_conti = pd.DataFrame(train,columns=["Survived","Pclass","Age","SibSp","Parch","Fare"],index=train.index)

In [23]:
train = train_conti.join(train_dummy)

In [24]:
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [25]:
X = train.iloc[:, 1:]
y = train.iloc[:, 0 ]

In [26]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [27]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      712 non-null    int64  
 1   Age         712 non-null    float64
 2   SibSp       712 non-null    int64  
 3   Parch       712 non-null    int64  
 4   Fare        712 non-null    float64
 5   Sex_female  712 non-null    uint8  
 6   Sex_male    712 non-null    uint8  
 7   Embarked_C  712 non-null    uint8  
 8   Embarked_Q  712 non-null    uint8  
 9   Embarked_S  712 non-null    uint8  
dtypes: float64(2), int64(3), uint8(5)
memory usage: 56.9 KB


In [28]:
X_train.head(5)

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
202,3,34.0,0,0,6.4958,0,1,0,0,1
439,2,31.0,0,0,10.5000,0,1,0,0,1
102,1,21.0,0,1,77.2875,0,1,0,0,1
118,1,24.0,0,1,247.5208,0,1,1,0,0
625,1,61.0,0,0,32.3208,0,1,0,0,1


In [29]:
stdsc = StandardScaler()
X_train_conti_std = stdsc.fit_transform(X_train[["Age","SibSp","Parch","Fare"]])
X_test_conti_std = stdsc.fit_transform(X_test[["Age","SibSp","Parch","Fare"]])

In [30]:
X_train_conti_std

array([[ 0.34867085, -0.54307331, -0.50118147, -0.50843012],
       [ 0.13760349, -0.54307331, -0.50118147, -0.43584459],
       [-0.56595439, -0.54307331,  0.64899536,  0.77483576],
       ...,
       [-0.91773333, -0.54307331, -0.50118147, -0.15487067],
       [ 0.70044979, -0.54307331, -0.50118147, -0.15487067],
       [-0.35488703, -0.54307331,  2.949349  , -0.27707982]])

In [31]:
X_train_conti_std = pd.DataFrame(data=X_train_conti_std,columns=[["Age","SibSp","Parch","Fare"]],index=X_train.index)
X_test_conti_std = pd.DataFrame(data=X_test_conti_std,columns=[["Age","SibSp","Parch","Fare"]],index=X_test.index)

In [32]:
X_train_conti_std

,Age,SibSp,Parch,Fare
202,0.348671,-0.543073,-0.501181,-0.508430
439,0.137603,-0.543073,-0.501181,-0.435845
102,-0.565954,-0.543073,0.648995,0.774836
118,-0.354887,-0.543073,0.648995,3.860714
625,2.248277,-0.543073,-0.501181,-0.040291
...,...,...,...,...
885,0.700450,-0.543073,5.249703,-0.098223
239,0.278315,-0.543073,-0.501181,-0.403669
791,-0.917733,-0.543073,-0.501181,-0.154871
705,0.700450,-0.543073,-0.501181,-0.154871


In [33]:
X_train

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
202,3,34.0,0,0,6.4958,0,1,0,0,1
439,2,31.0,0,0,10.5000,0,1,0,0,1
102,1,21.0,0,1,77.2875,0,1,0,0,1
118,1,24.0,0,1,247.5208,0,1,1,0,0
625,1,61.0,0,0,32.3208,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
885,3,39.0,0,5,29.1250,1,0,0,1,0
239,2,33.0,0,0,12.2750,0,1,0,0,1
791,2,16.0,0,0,26.0000,0,1,0,0,1
705,2,39.0,0,0,26.0000,0,1,0,0,1


In [34]:
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train,y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=0)

In [35]:
y_pred = classifier.predict(X_test)

In [36]:
y_pred

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0])

In [42]:
confusion_matrix = confusion_matrix(y_test,y_pred)

In [43]:
confusion_matrix

array([[105,  20],
       [ 29,  60]])

In [44]:
classifier.score(X_test,y_test)

<bound method ClassifierMixin.score of LogisticRegression(random_state=0)>